In [ ]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())

In [ ]:
import torch
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1,2,3])
tensor2d = torch.tensor([[1,2],[3,4]])
tensor3d = torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]])

In [ ]:
tensor1d = torch.tensor([1,2,3])
print(tensor1d.dtype)

In [ ]:
tensor1d = torch.tensor([1.0,2.0,3.0])
print(tensor1d.dtype)

In [ ]:
tensor1d = torch.tensor([1,2,3])
print(tensor1d.dtype)
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

In [ ]:
tensor2d = torch.tensor([[1,2,3],[4,5,6]])
print(tensor2d)
print(tensor2d.shape)
print(tensor2d.reshape(3,2))
print(tensor2d.view(3,2))
print(tensor2d.T)
print(tensor2d.matmul(tensor2d.T)) #矩阵乘法
print(tensor2d @ tensor2d.T) #矩阵乘法

In [21]:
import torch.nn.functional as F #A 这是PyTorch中常见的导入约定,用于避免代码行过长

y =  torch.tensor([1.0]) #B 真实标签
x1 = torch.tensor([1.1]) #C 输入特征
w1 = torch.tensor([2.2]) #D 权重参数
b = torch.tensor([0.0]) #E 偏置单元
z = x1 * w1 + b #F 网络输入
a = torch.sigmoid(z) #G 激活与输出

loss = F.binary_cross_entropy(a,y)

In [ ]:
# requires_grad=True是一个开关： 
# 你可以告诉 PyTorch，对于某些参与计算的数字（更专业地说，是“张量”），
# 你可能想知道它们是如何影响最终结果的。
# 如果你这样做了（通过设置 requires_grad=True），
# PyTorch 就会特别留意这些数字，并在计算图中记录下相关的操作。

import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss,w1,retain_graph=True)
grad_L_b = grad(loss,b,retain_graph=True)

# 默认情况下，PyTorch 在计算完梯度后会销毁计算图以释放内存。
# 然而，由于我们稍后将重用这个计算图，所以我们设置了 retain_graph=True，使其保留在内存中。

print(grad_L_w1)
print(grad_L_b)

# PyTorch 将计算图中所有叶节点的梯度，这些梯度将存储在张量的 .grad 属性中
loss.backward()
print(w1.grad)
print(b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor([-0.0898])
tensor([-0.0817])


In [33]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self,num_inputs,num_outputs): #A
        super().__init__()
        self.layers = torch.nn.Sequential(
            #1st hidden layer
            torch.nn.Linear(num_inputs,30), #B
            torch.nn.ReLU(), #C

            #2nd hidden layer
            torch.nn.Linear(30,20), #D
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(20,num_outputs),
        )

    def forward(self,x):
        logits = self.layers(x) #E
        return logits
    
#A 将输入和输出的数量编码为变量很有用，这样可以为具有不同特征和类别数量的数据集重用相同的代码。
#B Linear 层将输入和输出节点的数量作为参数。
#C 非线性激活函数放置在隐藏层之间。
#D 一个隐藏层的输出节点数必须与下一个隐藏层的输入节点数相匹配。
#E 最后一层的输出被称为 logits。

# model  = NeuralNetwork(50,3) #实例化一个新的神经网络对象
# print(model)

# # 检查模型的总可训练参数数量
# num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print("Total number of trainable model parameters:", num_params)

# # 第一个Linear层位于layers属性的索引位置0。
# # 我们可以按如下方式访问相应的权重参数矩阵
# print(model.layers[0].weight)
# print(model.layers[0].weight.shape)
# print(model.layers[0].bias)

# #如果希望每次运行代码时，这些随机数都是一样的，这样方便我们做实验和调试。
# #PyTorch提供了一个方法来实现这个目标，
# # 我们可以通过使用 manual_seed 来为 PyTorch 的随机数生成器设置种子，
# # 从而使随机数初始化可复现
# torch.manual_seed(123)
# model = NeuralNetwork(50, 3)
# print(model.layers[0].weight)

# #前项传播结果
# torch.manual_seed(123)
# X = torch.rand((1, 50))
# out = model(X)
# print(out)

# # 当我们将模型用于推理（例如，进行预测）而不是训练时，
# # 最佳实践是使用torch.no_grad()上下文管理器
# with torch.no_grad():
#     out = model(X)
# print(out)

# 在 PyTorch 中，一种常见的做法是将模型编码成返回最后一层（logits）的输出，
# 而不会将它们传递给非线性激活函数。
# 这是因为PyTorch 常用的损失函数将 softmax（或二元分类的sigmoid）运算与负对数似然损失组合在一个类中。
# 这样做的原因是出于数值效率和稳定性的考虑。
# 因此，如果我们想计算预测的类别成员概率，则必须显式调用softmax函数
with torch.no_grad():
    out = torch.softmax(model(X),dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])
